Well, looks like the Fraud-Detection competition got extended by a few days and people are salty. I'm not too invested in the competition so I'm not bothered by it, might as well play around with some more things before it ends. 

In this notebook I'm going to work on feature creation and selection. This is an area I have barely touched since going down the data science route, let's see where it takes us. 

In [6]:
import pandas as pd
import numpy as np
import time

In [22]:
t0 = time.time()

train_transaction = pd.read_csv('Data/train_transaction.csv')

t1 = time.time()

print(t1-t0)

51.340735912323


So the first thing I'm going to try to play around with is on how to deal with categorical variables. I was initially using the one hot/dummy encoding method, I'm going to try using more advanced methods, namely feature hashingand binning. I'll compare them to using dummy encoding. These methods should ideally save on memory, so this will be beneficial to me and my MacBook Air with 4GB of RAM.

In [23]:
strings = train_transaction.select_dtypes(include='object')
strings.fillna('NaN', inplace = True)
dummies = pd.get_dummies(strings)

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [39]:
from sklearn.feature_extraction import FeatureHasher
cols = []
for column in strings:
    classes = len(strings[column].unique())
    h = FeatureHasher(n_features=classes, input_type='string')
    col = h.transform(strings[column])
    col = col.toarray()
    cols.append(col)

In [58]:
print(cols)

[array([[ 0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       ...,
       [ 0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0., -1.]]), array([[ 1., -1., -1.,  0.,  1.],
       [ 4.,  1., -1.,  0.,  0.],
       [ 1., -1.,  0., -1., -1.],
       ...,
       [ 4.,  1., -1.,  0.,  0.],
       [ 4.,  1., -1.,  0.,  0.],
       [ 4.,  1., -1.,  0.,  0.]]), array([[ 1., -1., -1.,  0.,  1.],
       [ 1., -1., -1.,  0.,  1.],
       [ 0., -2.,  0.,  0.,  1.],
       ...,
       [ 0., -2.,  0.,  0.,  1.],
       [ 0., -2.,  0.,  0.,  1.],
       [ 1., -1., -1.,  0.,  1.]]), array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 

Ok, looks like hashing doesn't do what exactly I thought it would. Let's try factorizing instead. This isn't ideal in theory because it implies that the categories are somehow ordered, but we'll see how it fares in practice.

In [65]:
from sklearn.preprocessing import LabelEncoder as le
le = le()
for column in strings:
    le.fit(strings[column])
    le.transform(strings[column])

In [66]:
from sklearn.preprocessing import LabelEncoder

In [67]:
encoded = strings.apply(LabelEncoder().fit_transform)

In [68]:
encoded.head(10)

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,4,2,2,0,0,2,2,2,2,0,2,1,1,1
1,4,3,2,17,0,1,1,1,0,2,2,1,1,1
2,4,4,3,36,0,2,2,2,0,0,0,0,0,0
3,4,3,3,54,0,1,1,1,0,2,0,1,1,1
4,1,3,2,17,0,1,1,1,3,1,1,1,1,1
5,4,4,3,17,0,2,2,2,1,0,2,1,1,1
6,4,4,3,54,0,2,2,2,0,0,0,2,2,2
7,4,4,3,30,0,1,1,1,0,0,0,1,1,1
8,1,4,3,2,0,1,1,1,3,1,1,1,1,1
9,4,3,3,54,0,2,2,2,0,2,2,1,1,1
